# Interactions - Lab

## Introduction

In this lab, you'll explore interactions in the Ames Housing dataset.

## Objectives

You will be able to:
- Implement interaction terms in Python using the `sklearn` and `statsmodels` packages 
- Interpret interaction variables in the context of a real-world problem 

## Build a baseline model 

You'll use a couple of built-in functions, which we imported for you below: 

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

If you still want to build a model in the end, you can do that, but this lab will just focus on finding meaningful insights in interactions and how they can improve $R^2$ values.

In [3]:
regression = LinearRegression()

Create a baseline model which includes all the variables we selected from the Ames housing data set to predict the house prices. Then use 10-fold cross-validation and report the mean $R^2$ value as the baseline $R^2$.

In [4]:
ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_restricted = ames[continuous]
for var in categoricals:
    dummy = pd.get_dummies(ames[var], drop_first=True, prefix=var)
    ames_restricted = pd.concat([ames_restricted, dummy], axis=1)

X = ames_restricted.drop('SalePrice', axis=1)
y = ames_restricted['SalePrice']
ames_restricted.columns

Index(['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice', 'BldgType_2fmCon',
       'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE',
       'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA', 'SaleType_CWD',
       'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw',
       'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'MSZoning_FV',
       'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave',
       'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor',
       'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_IDOTRR',
       'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes',
       'Neighborhood_NPkVill', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge',
       'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU',
       'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst',
       'Neighborhood_StoneB

In [5]:
linreg = LinearRegression()
R2_base = np.mean(cross_val_score(linreg, X, y, cv=10,  scoring='r2'))

## See how interactions improve your baseline

Next, create all possible combinations of interactions, loop over them and add them to the baseline model one by one to see how they affect the $R^2$. We'll look at the 3 interactions which have the biggest effect on our $R^2$, so print out the top 3 combinations.

You will create a `for` loop to loop through all the combinations of 2 predictors. You can use `combinations` from itertools to create a list of all the pairwise combinations. To find more info on how this is done, have a look [here](https://docs.python.org/2/library/itertools.html).

Since there are so many different neighbourhoods we will exclude

In [6]:
from itertools import combinations
combs = combinations(X.columns, 2)
term_list = []
for comb in combs:
    comb0_name = comb[0]
    comb1_name = comb[1]
    if (comb0_name.find('_')  > 0) & (comb1_name.find('_') > 0):
        comb0_head = comb0_name.split('_')[0]
        comb1_head = comb1_name.split('_')[0]
        if comb0_head == comb1_head:
            continue
    Xtmp = X.copy()
    var_name = comb[0] + '_' + comb[1]
    Xtmp[var_name] = X[comb[0]]*X[comb[1]]
    term = {
        'comb': comb, 
        'R2': np.mean(cross_val_score(linreg, Xtmp, y, cv=10,  scoring='r2'))
    }
    print(term)
    term_list.append(term)

{'comb': ('LotArea', '1stFlrSF'), 'R2': 0.7987752731271455}
{'comb': ('LotArea', 'GrLivArea'), 'R2': 0.7979024765709419}
{'comb': ('LotArea', 'BldgType_2fmCon'), 'R2': 0.7969578208510304}
{'comb': ('LotArea', 'BldgType_Duplex'), 'R2': 0.7979010695968426}
{'comb': ('LotArea', 'BldgType_Twnhs'), 'R2': 0.7978299663727981}
{'comb': ('LotArea', 'BldgType_TwnhsE'), 'R2': 0.7975088338912408}
{'comb': ('LotArea', 'KitchenQual_Fa'), 'R2': 0.7978454783215152}
{'comb': ('LotArea', 'KitchenQual_Gd'), 'R2': 0.7838560381703769}
{'comb': ('LotArea', 'KitchenQual_TA'), 'R2': 0.7938986553089011}
{'comb': ('LotArea', 'SaleType_CWD'), 'R2': 0.7863185959734296}
{'comb': ('LotArea', 'SaleType_Con'), 'R2': 0.7982772230994849}
{'comb': ('LotArea', 'SaleType_ConLD'), 'R2': 0.7977263871422411}
{'comb': ('LotArea', 'SaleType_ConLI'), 'R2': 0.7980302275252764}
{'comb': ('LotArea', 'SaleType_ConLw'), 'R2': 0.7977779622379048}
{'comb': ('LotArea', 'SaleType_New'), 'R2': 0.7992094159044767}
{'comb': ('LotArea', 'Sa

KeyboardInterrupt: 

In [ ]:
# code to find top interactions by R^2 value here
top_interactions = sorted(term_list, key= lambda x: x['R2'], reverse=True)[:3]
top_interactions

In [ ]:
comb = top_interactions[1]['comb']

Xextended = X.copy()
var_name = comb[0] + '_' + comb[1]
Xextended[var_name] = X[comb[0]]*X[comb[1]]
linreg.fit(Xextended, y)

It looks like the top interactions involve the Neighborhood_Edwards feature so lets add the interaction between LotArea and Edwards to our model.

We can interpret this feature as the relationship between LotArea and SalePrice when the house is in Edwards or not.

## Visualize the Interaction

Separate all houses that are located in Edwards and those that are not. Run a linear regression on each population against `SalePrice`. Visualize the regression line and data points with price on the y axis and LotArea on the x axis.

In [ ]:
# Visualization code here
edwards = ames_restricted.query('Neighborhood_Edwards == 1')
other = ames_restricted.query('Neighborhood_Edwards == 0')

In [ ]:
print(len(edwards), len(other))

In [ ]:
other.head()

In [ ]:
Xed = edwards['LotArea'].values.reshape(-1, 1)
yed = edwards['SalePrice'].values.reshape(-1, 1)
Xot = other['LotArea'].values.reshape(-1, 1)
yot = other['SalePrice'].values.reshape(-1, 1)

plt.figure(figsize=(8,6))
edReg = linreg.fit(Xed, yed)
plt.scatter(Xed, yed, color='blue', alpha=0.2)
otReg = linreg.fit(Xot, yot)
plt.scatter(Xot, yot, color='red', alpha=0.2)

edPred = edReg.predict(Xed)
plt.plot(Xed, edPred, color='blue')
otPred = otReg.predict(Xot)
plt.plot(Xot, otPred, color='red')

## Build a final model with interactions

Use 10-fold cross-validation to build a model using the above interaction. 

In [7]:
# code here

regression = LinearRegression()
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
final = X.copy()

final['Neighborhood_Edwards*LotArea'] = final['Neighborhood_Edwards'] * final['LotArea']

final_model = np.mean(cross_val_score(regression, final, y, scoring='r2', cv=crossvalidation))

final_model

0.8093314939295162

Our $R^2$ has increased considerably! Let's have a look in `statsmodels` to see if this interactions are significant.

In [8]:
# code here
import statsmodels.api as sm
df_inter_sm = sm.add_constant(final)
model = sm.OLS(y,final)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              SalePrice   R-squared (uncentered):                   0.973
Model:                            OLS   Adj. R-squared (uncentered):              0.972
Method:                 Least Squares   F-statistic:                              1061.
Date:                Wed, 30 Sep 2020   Prob (F-statistic):                        0.00
Time:                        16:34:10   Log-Likelihood:                         -17238.
No. Observations:                1460   AIC:                                  3.457e+04
Df Residuals:                    1412   BIC:                                  3.482e+04
Df Model:                          48                                                  
Covariance Type:            nonrobust                                                  
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
LotArea                          0.6528      0.103      6.323      0.000       0.450       0.855
1stFlrSF                        37.4553      3.250     11.525      0.000      31.080      43.831
GrLivArea                       59.0489      2.407     24.530      0.000      54.327      63.771
BldgType_2fmCon              -1.751e+04   6387.159     -2.741      0.006      -3e+04   -4976.763
BldgType_Duplex              -3.219e+04   4965.678     -6.483      0.000   -4.19e+04   -2.25e+04
BldgType_Twnhs               -3.219e+04   6883.170     -4.677      0.000   -4.57e+04   -1.87e+04
BldgType_TwnhsE              -1.792e+04   4360.335     -4.110      0.000   -2.65e+04   -9366.122
KitchenQual_Fa               -7.605e+04   7071.289    -10.754      0.000   -8.99e+04   -6.22e+04
KitchenQual_Gd               -5.052e+04   4009.263    -12.602      0.000   -5.84e+04   -4.27e+04
KitchenQual_TA               -6.567e+04   4411.601    -14.886      0.000   -7.43e+04    -5.7e+04
SaleType_CWD                  2.467e+04   1.74e+04      1.418      0.156   -9452.155    5.88e+04
SaleType_Con                  6.674e+04   2.46e+04      2.716      0.007    1.85e+04    1.15e+05
SaleType_ConLD                2.717e+04   1.23e+04      2.214      0.027    3096.978    5.12e+04
SaleType_ConLI                2.359e+04   1.57e+04      1.500      0.134   -7249.780    5.44e+04
SaleType_ConLw                1.629e+04   1.58e+04      1.033      0.302   -1.46e+04    4.72e+04
SaleType_New                  3.845e+04   6195.216      6.207      0.000    2.63e+04    5.06e+04
SaleType_Oth                  1.327e+04   1.98e+04      0.669      0.503   -2.56e+04    5.21e+04
SaleType_WD                   1.772e+04   5136.521      3.449      0.001    7640.513    2.78e+04
MSZoning_FV                   6.257e+04   1.43e+04      4.364      0.000    3.44e+04    9.07e+04
MSZoning_RH                   4.314e+04   1.44e+04      2.998      0.003    1.49e+04    7.14e+04
MSZoning_RL                   5.459e+04   1.14e+04      4.768      0.000    3.21e+04    7.71e+04
MSZoning_RM                   5.383e+04   1.12e+04      4.802      0.000    3.18e+04    7.58e+04
Street_Pave                   2.947e+04   1.22e+04      2.421      0.016    5588.653    5.33e+04
Neighborhood_Blueste          2.888e+04   2.51e+04      1.149      0.251   -2.04e+04    7.82e+04
Neighborhood_BrDale           8501.3381   1.28e+04      0.665      0.506   -1.66e+04    3.36e+04
Neighborhood_BrkSide         -2.152e+04   9797.097     -2.197      0.028   -4.07e+04   -2305.128
Neighborhood_ClearCr         -8162.5278   1.07e+04     -0.764      0.445   -2.91e+04    1.28e+04
Neighborhood_CollgCr          8563.6560   8458.465      1.012      0.312  

What is your conclusion here?

In [ ]:
# formulate your conclusion

## Summary

You should now understand how to include interaction effects in your model! As you can see, interactions can have a strong impact on linear regression models, and they should always be considered when you are constructing your models.